In [1]:
import nltk
nltk.download('punkt')

import pandas as pd
from nltk.tokenize import word_tokenize
from csv import QUOTE_NONE

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\raghu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Question 1

In [2]:
import numpy as np

In [3]:
df_sst = pd.read_csv("SST-2/train.tsv",delimiter="\t")
df_sst.head(3)
df_sst_c=pd.DataFrame(df_sst['sentence'])

In [4]:
df_sst

,sentence,label
0,hide new secretions from the parental units,0
1,"contains no wit , only labored gags",0
2,that loves its characters and communicates som...,1
3,remains utterly satisfied to remain the same t...,0
4,on the worst revenge-of-the-nerds clichés the ...,0
...,...,...
67344,a delightful comedy,1
67345,"anguish , anger and frustration",0
67346,"at achieving the modest , crowd-pleasing goals...",1
67347,a patient viewer,1


In [15]:
df_sst1 = df_sst.sample(frac=1).reset_index()
print(df_sst1)

       index                                           sentence  label
0      53858  its and pieces of the hot chick are so hilarious       1
1      32877   stillborn except as a harsh conceptual exercise       0
2      61990  flat-out amusing , sometimes endearing and oft...      1
3      42737  is about the people who live in them , who hav...      1
4      49433           rarely easy , obvious or self-indulgent       1
...      ...                                                ...    ...
67344  50781                                       a real film       1
67345  22851  to ponder how a whole segment of pop-music his...      0
67346  25006  sand 's masculine persona , with its love of l...      1
67347   6325                                     nutty cliches       0
67348  18584                                           goodies       1

[67349 rows x 3 columns]


In [16]:
test_df=df_sst1[:100]

In [20]:
validation_df=df_sst1[100:200]

In [24]:
train_df=df_sst1[200:]

In [28]:
dist_label=train_df.label.value_counts()

In [36]:
dict(dist_label)

{1: 37461, 0: 29688}

In [37]:
tot_label=len(train_df)

In [51]:
prior_prob_1=dist_label[1]/tot_label

In [52]:
prior_prob_0=dist_label[0]/tot_label

In [53]:
print("prior probablity of class 1 : ",prior_prob_1)
print("prior probablity of class 0 : ",prior_prob_0)

prior probablity of class 1 :  0.5578787472635482
prior probablity of class 0 :  0.44212125273645175


## Question 2

In [55]:
# train_df

In [392]:
def convert_tokenizer(sent):
    tkns=word_tokenize(sent)
    tkns = ['<s>'] + tkns + ['</s>']
    return tkns


    

In [393]:
print("first line of train data tokanization")
print(convert_tokenizer(train_df.sentence[200]))

first line of train data tokanization
['<s>', 'the', 'pacing', 'is', 'often', 'way', 'off', 'and', 'there', 'are', 'too', 'many', 'bona', 'fide', 'groaners', 'among', 'too', 'few', 'laughs', '.', '</s>']


In [394]:
all_sent_train=[]
for i in train_df.sentence:
    all_sent_train.append(convert_tokenizer(i))

In [395]:
tokens = [token for sentence_tokens in all_sent_train for token in sentence_tokens]
    

In [396]:
len(set(tokens))

14806

In [401]:
tokeni_dict={}
for i in tokens:
    if(i not in tokeni_dict.keys()):
        tokeni_dict[i]=1
    else:
        tokeni_dict[i]=tokeni_dict[i]+1

In [420]:

len(set(tokens))

14806

## Question 3

In [243]:
def bigram_frequencies(all_sent_train):
    bc = {}  
    
    for s in all_sent_train:
        for i in range(len(s) - 1):
            wi = s[i] 
            wj=s[i + 1] 
            

            if wi not in bc:
                bc[wi] = {}

            if wj not in bc[wi]:
                bc[wi][wj] = 0

            bc[wi][wj] += 1

    return bc


In [106]:
bigram_counts = bigram_frequencies(all_sent_train)

In [114]:
q3_answer = bigram_counts.get("<s>").get("the")


In [115]:
q3_answer

4456

In [116]:
len(bigram_counts.keys())

14805

In [124]:
import numpy as np

## Question 4

In [355]:
import math
def smooth_neg_prob(wn,wn_pre,bc,alpha,voc):
    try:
        
        cnt=bc.get(wn_pre).get(wn,0)
    except:
        cnt=0
    sum_cnt=np.array(list(bigram_counts.get(wn_pre,0).values())).sum()
    p=(cnt+alpha)/(sum_cnt+voc*alpha)

    return -1*math.log(p)

In [169]:
print("alpha 0.001")
smooth_neg_prob("award","academy",bigram_counts,0.001,len(bigram_counts.keys()))

alpha 0.001


1.024899084055268

In [170]:
print("alpha 0.5")
smooth_neg_prob("award","academy",bigram_counts,0.5,len(bigram_counts.keys()))

alpha 0.5


6.172373816771201

## Question 5

In [352]:
def smooth_neg_prob_sent(sent_prom,bc,alpha,voc):
    tot_log_prob=0
    tok_sent=nltk.word_tokenize(sent_prom)
    for i in range(1,len(tok_sent)):
        temp_log=smooth_neg_prob(tok_sent[i],tok_sent[i-1],bc,alpha,voc)
        tot_log_prob=tot_log_prob-temp_log
    return tot_log_prob
        
        

In [198]:
sent_prom_1="this was a really great movie but it was a little too long."
sent_prom_2="long too little a was it but movie great really a was this."
alpha1=0.5
alpha2=0.5


In [199]:
print("for sentence 1")
smooth_neg_prob_sent(sent_prom_1,bigram_counts,alpha1,len(bigram_counts.keys()))

for sentence 1


-78.68998057967534

In [200]:
print("for sentence 2")
smooth_neg_prob_sent(sent_prom_2,bigram_counts,alpha2,len(bigram_counts.keys()))

for sentence 2


-116.84105640459761

## Question 6

In [205]:
validation_snt=""
for i in list(validation_df.sentence):
    validation_snt=validation_snt+i

In [220]:
al1=[0.001,0.01,0.1]
best_alpha={}

for i in al1:
    
    log_ans=smooth_neg_prob_sent(validation_snt,bigram_counts,i,len(bigram_counts.keys()))
    best_alpha[log_ans]=i
    print("alpha",i,log_ans)

alpha 0.001 -3697.1190755623497
alpha 0.01 -3911.427647317069
alpha 0.1 -4534.357858886998


In [226]:
print("the best alpha value is",best_alpha[max(best_alpha.keys())],"of",max(best_alpha.keys()))

the best alpha value is 0.001 of -3697.1190755623497


## Question 7

In [235]:
train_df_1=train_df.loc[train_df.label==1]
train_df_0=train_df.loc[train_df.label==0]

In [238]:
all_sent_train_p=[]
for i in train_df_1.sentence:
    all_sent_train_p.append(convert_tokenizer(i))
    
all_sent_train_n=[]

for i in train_df_0.sentence:
    all_sent_train_n.append(convert_tokenizer(i))

In [239]:
tokens_p = [token for sentence_tokens in all_sent_train_p for token in sentence_tokens]
tokens_n = [token for sentence_tokens in all_sent_train_n for token in sentence_tokens]

In [241]:
print(len(set(tokens_p)))
print(len(set(tokens_n)))

11524
11240


In [244]:
bigram_counts_p = bigram_frequencies(all_sent_train_p)
bigram_counts_n = bigram_frequencies(all_sent_train_n)

In [256]:
test_df_pred=pd.DataFrame()
prediction=np.zeros(100)

In [260]:
test_df['pred']=prediction

C:\Users\raghu\AppData\Local\Temp\ipykernel_35744\759525314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['pred']=prediction


In [360]:
p_prior=len(bigram_counts_p)/(len(bigram_counts_p)+len(bigram_counts_n))
n_prior=len(bigram_counts_n)/(len(bigram_counts_p)+len(bigram_counts_n))
ans_pred=[]
for i in range(0,len(test_df)):
    l1=test_df[i:i+1]
    temp_sent=list(l1['sentence'])[0]
    ps=smooth_neg_prob_sent(temp_sent,bigram_counts_p,0.001,len(bigram_counts_p))
    ns=smooth_neg_prob_sent(temp_sent,bigram_counts_n,0.001,len(bigram_counts_n))

    final_p=ps-p_prior
    final_n=ns-n_prior
    if(final_p>final_n):
        ans_pred.append(1)
    else:
        ans_pred.append(0)
        
    
        
        
    
    

# 

In [361]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(list(test_df.label), ans_pred)

In [362]:
print("the accuracy is ",accuracy)

the accuracy is  0.9


In [366]:
from sklearn.metrics import precision_score, recall_score

In [371]:
print("the precision score is")
precision_score(list(test_df.label), ans_pred)

the precision score is


0.9259259259259259

In [372]:
print("the recall score is")
recall_score(list(test_df.label), ans_pred)

the recall score is


0.8928571428571429